<a href="https://colab.research.google.com/github/DonnaVakalis/Personal/blob/master/DJW_slack_word.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

For the code in this notebook, I am indebted to:


*   https://www.analyticsvidhya.com/blog/2018/02/the-different-methods-deal-text-data-predictive-python/
*   https://stackoverflow.com/questions/56866697/python-read-json-files-from-all-sub-directories



 

In [0]:
# IMPORTS

import os
import glob
import pandas as pd
from pathlib import Path

In [3]:
# LOCATE DATA FILES

from google.colab import drive
drive.mount('/content/gdrive')



Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive/RawDat


In [7]:
os.chdir("/content/gdrive/My Drive/RawDat/Slack_export_June8")
working_directory = os.getcwd()
#print(working_directory)
sub_directories = [working_directory + "/" + x for x in os.listdir(working_directory) if os.path.isdir(working_directory + "/"+x)]
all_json_files = []

for sub_dir in sub_directories:
    os.chdir(sub_dir)
    for file in glob.glob("*.json"):
        all_json_files.append(sub_dir + "/" + file)

#Get back to original working directory
os.chdir(working_directory)

list_of_dfs = [pd.read_json(x) for x in all_json_files]

ValueError: ignored

In [0]:
# GET DAT

files = Path("C:/Users/Donna/Push_to_Git/HousingLCA/RawData_doNotUpload/Slack_export_June8").glob("**/*.json")

for filename in files:
    df = pd.read_json(filename)
    print(df)

In [8]:
files

<generator object Path.glob at 0x7f7dd75de938>

In [0]:
# Get data

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving train_E6oV3lV.csv to train_E6oV3lV (1).csv
User uploaded file "train_E6oV3lV.csv" with length 3103165 bytes
